In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import os
import shutil
from sklearn.model_selection import train_test_split

In [3]:
# Set Image directories
dataset_dir = '../dataset/images'
train_dir = '../dataset/train'
validation_dir = '../dataset/val'
test_dir = '../dataset/test'

In [ ]:
# Function to delete and recreate directories
def reset_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory, exist_ok=True)

# Reset train, validation, and test directories
reset_directory(train_dir)
reset_directory(val_dir)
reset_directory(test_dir)

# Get all subdirectories (each representing a dog breed)
breed_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]

print(f'Number of breeds: {len(breed_dirs)}')

# Iterate through each breed directory
for breed in breed_dirs:
    breed_path = os.path.join(dataset_dir, breed)
    images = [os.path.join(breed_path, img) for img in os.listdir(breed_path) if img.endswith(('jpg', 'jpeg', 'png'))]

    # Split the images into train, validation, and test sets
    train_images, temp_images = train_test_split(images, test_size=0.3, random_state=42)
    val_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42)

    # Create breed directories in train, validation, and test directories
    os.makedirs(os.path.join(train_dir, breed), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, breed), exist_ok=True)
    os.makedirs(os.path.join(test_dir, breed), exist_ok=True)

    # Function to copy images to respective directories
    def copy_images(image_list, target_dir):
        for image_path in image_list:
            shutil.copy(image_path, os.path.join(target_dir, breed))

    # Copy images to train, validation, and test directories
    copy_images(train_images, train_dir)
    copy_images(val_images, validation_dir)
    copy_images(test_images, test_dir)

Number of breeds: 120


In [ ]:
# Data generator functions
def create_datagen(preprocessing_function =""):
    return ImageDataGenerator(
        preprocessing_function=preprocessing_function,
        rescale=1./255,  # Normalize pixel values
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
    )
def create_generator(datagen, directory, shuffle):
    return datagen.flow_from_directory(
        directory,
        target_size=(224,224),
        batch_size=16,
        class_mode='categorical',
        shuffle=shuffle,
    )

In [ ]:
# Create data generators
datagen_nasnet = create_datagen(tf.keras.applications.vgg16.preprocess_input)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator_nasnet = create_generator(datagen_nasnet, train_dir, True)
val_generator = create_generator(val_datagen, validation_dir, True)
test_generator = create_generator(test_datagen, test_dir, False)

In [ ]:
# Model Building
nasnet_model = tf.keras.applications.NASNetLarge(weights='imagenet', include_top=False, input_shape=(224,224,3))
x = nasnet_model.output
# Freeze base model layers
nasnet_model.trainable = False
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x_preds = tf.keras.layers.Dense(120, activation='softmax')(x)
nasnet_model = tf.keras.Model(inputs=nasnet_model.input, outputs=x_preds)

In [ ]:
nasnet_model.summary()

In [ ]:
nasnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Model Training
history = nasnet_model.fit(train_generator_nasnet, validation_data=val_generator, epochs=20)

In [ ]:
# Evaluate the model on the test data
test_loss, test_acc = nasnet_model.evaluate(test_generator, steps=test_generator.classes.size)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc}")

# Reset the test generator
test_generator.reset()

# Get the predictions
predictions = nasnet_model.predict(test_generator, steps=test_generator.classes.size)

# Get true labels
y_true = test_generator.classes

# Convert predicted probabilities to class labels
threshold = 0.5
y_pred = (predictions > threshold).astype(int).flatten()

# Print classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, zero_division=1))

# Print confusion matrix with labels
conf_matrix = confusion_matrix(y_true, y_pred)
conf_matrix_flipped = conf_matrix[[1, 0], :]  # Flip the rows
conf_matrix_flipped = conf_matrix_flipped[:, [1, 0]]  # Flip the columns
conf_matrix_df = pd.DataFrame(conf_matrix_flipped, index=["True BMR", "True Non-BMR"], columns=["Predicted BMR", "Predicted Non-BMR"])
print("Confusion Matrix:")
print(conf_matrix_df)

# Set the size of the plot
plt.figure(figsize=(14, 8))

# Set the font scale
sns.set(font_scale=2)

sns.heatmap(conf_matrix_df, annot=True, fmt='d', cmap='Blues', cbar_kws={'label': 'Count'})

plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')

plt.tight_layout()

plt.show()

# Calculate accuracy from confusion matrix
accuracy = np.trace(conf_matrix) / np.sum(conf_matrix)
print(f"Calculated Accuracy from Confusion Matrix: {accuracy * 100:.2f}%")

In [ ]:
# Plot training and validation accuracy
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Val Accuracy', marker='o')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.grid(True)
plt.show()

# Plot training and validation loss
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Train Loss', marker='o')
plt.plot(history.history['val_loss'], label='Val Loss', marker='o')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='best')
plt.grid(True)
plt.show()